In [1]:
import requests 
import time
import re
from datetime import datetime
from bs4 import BeautifulSoup,NavigableString, Comment

In [2]:
def convert_string(input_str):
    # Regular expression pattern to match a date in the format dd-mm-yyyy
    pattern = r'\b\d{2}/\d{2}/\d{4}\b'
    
    # Search for the pattern in the string
    match = re.search(pattern, input_str)
    if match:
    # Extract and print the date if found
        date_str = match.group(0)
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        date =  date_obj.strftime('%Y-%m-%d')

        return date
    else:
        'No date found'    
#convert time string to datetime object
def convert_time_string(posted_date):
    # Regular expression pattern to match a date in the format dd-mm-yyyy
    pattern = r'\b\d{2}/\d{2}/\d{4}\b'

    # Search for the pattern in the string and extract the date
    match = re.search(pattern, posted_date)
    if match:
        date_str = match.group(0)
        # Convert the extracted date string to datetime object
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        return date_obj
    else:
        return None 

In [3]:
response = requests.get('https://bongda24h.vn/hau-truong-c188-p1.html')
soup = BeautifulSoup(response.content, 'html.parser')

In [8]:
featured_posts = soup.find('div', class_ = 'post-featured')
urls = []
for i in featured_posts:
    path = i.find('h2').find('a')['href']
    url = 'https://bongda24h.vn/' + path
    urls.append(url)
section_content = soup.find_all('div

In [9]:
urls

['https://bongda24h.vn//tin-nong/miss-golf-le-thanh-tu-va-bai-hoc-tu-huyen-thoai-golf-279-371679.html',
 'https://bongda24h.vn//hau-truong/filip-nguyen-tiet-lo-chuyen-tinh-voi-nguoi-ban-doi-xinh-dep-188-371621.html',
 'https://bongda24h.vn//tin-nong/hong-son-kim-hue-xuat-hien-tai-vinpearl-dic-legends-vietnam-2023-279-371605.html',
 'https://bongda24h.vn//hau-truong/hanh-dong-am-ap-cua-van-hau-voi-doan-hai-my-trong-tiec-cuoi-188-371556.html']